In [1]:
from pprint import pprint
from lxml import html
import requests
from IPython.display import Image, display
from pymongo import MongoClient

In [2]:
site = 'https://lenta.ru'

def getNews(site):
    r = requests.get(site)
    return r.content

def parseNews(content):
    res = []
    root = html.fromstring(content, parser = html.HTMLParser(encoding='utf-8'))
    sc = root.xpath("//div[@class='layout__container']/main[@class='layout__content js-site-container']/div[@class='main-page']/section[@class='main-page__section']")

    for section in sc:
        longgrid = section.xpath(".//div[@class='main-container']/div[@class='longgrid-list']")
        # columns
        for news_column in longgrid:
            box = news_column.xpath(".//div[@class='longgrid-list__box']")
            for a in box:            
                for news in a.xpath(".//a[@class='card-big _longgrid']"):
                    href = news.xpath(".//@href")[0]
                    title = news.xpath("div[@class='card-big__titles']/h3/text()")[0]
                    img = news.xpath("div[@class='card-big__image-wrap']/img/@src")[0]
                    date = news.xpath("div[@class='card-big__info']/time/text()")[0]
                    res.append({"href": href, "title": title, "date": date, "img": img})
                for news in a.xpath(".//a[@class='card-mini _longgrid']"):
                    href = news.xpath(".//@href")[0]
                    title = news.xpath("div[@class='card-mini__text']/span[@class='card-mini__title']/text()")[0]
                    date = news.xpath("div[@class='card-mini__text']/div[@class='card-mini__info']/time/text()")[0]
                    res.append({"href": href, "title": title, "date": date, "img": None})
    return res

def storeNewsToDB(news):    
    db = client.news
    db.news.drop()
    db.news.insert_many(news)
    
def printFromDbByDate(value):
    db = client.news
    r = db.news.find({ "date": { "$regex": value, "$options" :'i' } })
    for a in r:
        print(a["title"],a["date"])   



In [3]:
client = MongoClient('mongodb://localhost:27017/')
news = parseNews(getNews(site))
storeNewsToDB(news)
printFromDbByDate('3 января')

«Или вместе — или никто» 00:01, 3 января 2023
«Был выбор: победить или умереть» 00:01, 3 января 2023
«Кровожадненькая, но до чего же ламповая» 00:01, 3 января 2023
«Стошнит от одной мысли о таком» 00:02, 3 января 2023
